! jt -r

from importlib import reload
reload(aux)
reload(con)
reload(info)

In [95]:
from importlib import reload
import pandas as pd
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime,date
import src.info as info
import src.in_out as io
import numpy as np
import itertools

In [94]:
reload(con)

<module 'src.condition' from 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\src\\condition.py'>

# Hazırlık Aşaması

**data** klasöründeki tüm dosya yolları alınıyor
```python
>>> path_list
Out []:
 ['C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6A_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6B_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6C_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6A_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6B_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6C_U8.xlsx']
```

In [2]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

Ürün adları sırayla alınıyor
```py
>>> product_names
Out[]:
['6AU8', '6BU8', '6CU8', '6AU8', '6BU8', '6CU8']```

In [3]:
product_names = [info.get_productName(path) for path in path_list]

Tarihler sırayla alınıyor
```py
>>> date_list
Out[]:
[Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-07 00:00:00'),
 Timestamp('2018-09-07 00:00:00'),
 Timestamp('2018-09-07 00:00:00')]
```

In [4]:
info.get_productDate('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6A_U8.xlsx').date()

datetime.date(2018, 9, 6)

In [5]:
date_list = [info.get_productDate(path) for path in path_list]

Hangi sütunlardaki verileri alacağımızı aşağıdaki listede belirityoruz

In [6]:
cols= ["Time","BID price","ASK price"]

Dosyalardan veriler dataframe olarak okunuyor ve listeye atılıyor.
Okunan_veriler'in yapısı: <br/>
```py
list(pd.DataFrame,pd.DataFrame.....)
```

In [7]:
okunan_veriler = [pd.read_excel(path,usecols=cols) for path in path_list]

Her bir DataFrame'deki sütunlar yeniden adlandırılıyor

In [8]:
columns=['time','bid_price','ask_price']

In [9]:
for df in okunan_veriler:
    df.columns = columns

In [10]:
time_stamp = okunan_veriler[0].time

# Mid Price Hesaplama ve Saatlik Bölme 

hesaplanan her bir mid_price **list_of_mid_price** adlı listeye atılıyor. Listenin yapısı aşağıdaki gibidir:
```py
list(pd.Series,pd.Series,pd.Series...)
```

In [11]:
list_of_mid_price = list(map(aux.get_mid_price,okunan_veriler))

Bulunan mid price hiyerarşik indeksleme ile tek DataFrame'de birleştiriliyor.

In [12]:
arrays = [date_list,product_names]

In [13]:
df_mp = pd.DataFrame(data=list_of_mid_price, index = arrays)

Bu dataframe zaman sütunu eklenip saatlik olarak bölünüyor.

In [14]:
df_mp_t = df_mp.T

In [15]:
df_mp_t['time'] = time_stamp

```py
>>> len(list_of_hourly_mp)
Out[]: 23
>>> df.columns.levels
Out[]:FrozenList([[2018-09-06 00:00:00, 2018-09-07 00:00:00, 'time'], ['6AU8', '6BU8', '6CU8', '']])
```

In [16]:
list_of_hourly_mp = aux.split_df(df_mp_t,23)

----

# Ürünler Eşleştiriliyor

```python
>>> non_duplicate_dates
Out[]:[Timestamp('2018-09-06 00:00:00'), Timestamp('2018-09-07 00:00:00')]
>>> non_duplicate_names
Out[]:['6AU8', '6BU8', '6CU8']
>>> paired_products
Out[]: [('6AU8', '6BU8'), ('6AU8', '6CU8'), ('6BU8', '6CU8')]
```

In [17]:
non_duplicate_dates = list(dict.fromkeys(date_list)) #tekrar eden tarihler kaldırıldı.

In [18]:
non_duplicate_names = list(dict.fromkeys(product_names)) #tekrar eden ürünler kaldırıldı.

In [19]:
paired_products = list(itertools.combinations(non_duplicate_names, 2)) #2'li kombinasyon kullanılarak ürünler eşleştiriliyor.

----

# Spread Hesaplanıyor

**list_of_hourly_mp** da hiyerarşi şu şekildedir: <br/>
* Önce saate erişiriz
* İkinci olarak tarihe erişiriz
* Son olarak ürüne erişiriz <br/>
**Buna göre spread hesaplamak için aşağıdaki yol izlenir:** <br/>
    1. Her bir saate eriş
    2. Her bir tarihe eriş
    3. Pairleri al
    4. *spread_hesapla(pair): return pd.Series*
    

In [20]:
values={'a_PNLTICK':10,
'a_TICKSIZE':0.0001,
'b_PNLTICK':6.25,
'b_TICKSIZE':0.0001}

In [21]:
def spread_al(df):
    spread_dict = {}
    for date in non_duplicate_dates:
        for pair in paired_products:
            mp_1 = df[pd.Timestamp(date)][pair[0]]
            mp_2 = df[pd.Timestamp(date)][pair[1]]
            spread_dict[(date,pair[0]+"_"+pair[1])] = aux.find_spread(mp_1,mp_2,values)
    new_df = pd.DataFrame(spread_dict)
    new_df['time'] = df.time
    return new_df

In [22]:
spread = [spread_al(df) for df in list_of_hourly_mp]

----

# Change Hesaplanıyor

In [23]:
pairs = [pair[0]+'_'+pair[1] for pair in paired_products]

In [24]:
def change_al(df):
    change_dict = {}
    for date in non_duplicate_dates:
        for pair in pairs:
            sub_df = df[date][pair]            
            change_dict[(date,pair)] = aux.find_change(sub_df)
    new_df = pd.DataFrame(change_dict)
    new_df['time'] = df.time
    return new_df

In [25]:
change_list = [change_al(df) for df in spread]

---

# Duration ve Amplitude Hesaplanıyor

Hesaplama ile birlikte birleştirme de yapılıyor.
Birleştirme koşulu: <br/>
1. Ürünler aynı olacak
2. Tarihler farklı olacak
3. Saat dilimi aynı olacak

In [26]:
def unnamed1(df):
    lengt = df.shape[1]
    df_list = []
    for i in range(lengt):
        df_list.append(aux.find_duramp(df.iloc[:,1]))
    return pd.concat(df_list).dropna()

In [27]:
def unnamed2(df):
    new_df = duzenle(df)
    product = {}
    for pair in pairs:
        product[pair] = unnamed1(new_df[pair])
    return product

In [28]:
def duzenle(df):
    new_df = df.set_index('time')
    new_df = new_df.T.reset_index(level=0,drop=True).T
    return new_df

In [29]:
duramp = [unnamed2(change) for change in change_list]

---

# Medyanı baz alarak verileri yeniden düzenliyoruz

**pozitif ve negatif medyan, duration ve amplitude için ayrı ayrı baz alınarak hesaplanmıştır**
```py
>>> edited_data.keys()
Out[]: dict_keys(['6AU8_6BU8', '6AU8_6CU8', '6BU8_6CU8'])
```
Bir ürüne ait tüm veriye erişmek için:
```py
edited_data['6AU8_6BU8']
```
Bir ürünün belirli bir saatine ait veriye erişmek için:
```py
edited_data['6AU8_6BU8'].loc[time(18)]
```

In [97]:
hour_series = pd.date_range('2018-01-01-18', periods=23, freq='H').time

-------------

In [98]:
edited_data = {}

In [99]:
unconditional = {}

In [102]:
percentile_data = {}

In [103]:
for pair in pairs:
    con_list = []
    con_percent_list=[]
    uncon_list = []
    for i in range(23):
        current = duramp[i][pair]
        con_df = con.single_scan(current)
        con_df_percent = con.single_scan(current,condition='percentile',quantile_value=.25)
        con_list.append(con_df)
        uncon_list.append(current)
        con_percent_list.append(con_df_percent)
    edited_data[pair] = pd.concat(con_list,keys=hour_series)
    unconditional[pair] = pd.concat(uncon_list,keys= hour_series)
    percentile_data[pair] = pd.concat(con_percent_list,keys= hour_series)

---

# Dosyaya Yazılıyor

In [101]:
for pair in pairs:
        print("dosyaya yazılıyor: ",pair)
        df = edited_data[pair].reset_index()
        uncon_df = unconditional[pair].reset_index()
        df.columns=['time_period','time','duration','amplitude']
        uncon_df.columns=['time_period','time','duration','amplitude']
        df.to_csv(pair+".csv")
        uncon_df.to_csv(pair+"_unconditional.csv",index = False)

dosyaya yazılıyor:  6AU8_6BU8
dosyaya yazılıyor:  6AU8_6CU8
dosyaya yazılıyor:  6BU8_6CU8
